# <center>CHOOSING PRIOR 3</center>

---
# Norm $\propto$ norm $\times$ norm (biết variance của  data)

**Công thức:**  
Posterior N<font size="+1">$\left(\frac{n}{n+\frac{\sigma^2_0}{s^2_0}}\overline{X} + \frac{\frac{\sigma^2_0}{s^2_0}}{n+\frac{\sigma^2_0}{s^2_0}}m_0, \frac{\sigma^2_0}{n+\frac{\sigma^2_0}{s^2_0}}  \right) \propto $</font>
Likelihood N$\left( \mu, \sigma^2_0       \right) \times $
Prior N$\left( m_0, s^2_0       \right)  $

Trong đó:
- Các số $\sigma_0, m_0, s_0 $ là biết trước
- $\overline{X}$ là trung bình của data, tức là $\mu$

### Từ posterior mean, ta thấy:
- posterior mean = data *weight* x data mean + sample *weight* x sample mean
- n là data size, $\frac{\sigma^2_0}{s^2_0}$ là effective sample size
- effective sample size là tỉ số giữa variance data và variance prior

### Công thức trên hợp lý, thực vậy:
- Khi $s_0$ lớn, tức prior variance cao, prior không chứa thông tin nhiều
- Sample weight nhỏ so với data weight

### Prior predictive distribution:
- $\int^{\infty}_{-\infty} f(x|\mu)f(\mu)d\mu$
- Có thể nói một data mới $x = \mu + \epsilon, \mu = m_0 +\eta$
- Với $\epsilon \sim N(0, s^2_0), \eta \sim N(0,\sigma^2_0)$
- Vậy:
    - $ x \sim N(m_0, s^2_0 + \sigma^2_0)$
    
### Posterior predictive distribution:  
Tương tự với trên, ta có:  
x $\sim$ N(posterior mean, posterior variance + data variance)


---
# Norm $\propto$ norm $\times$ norm (không biết variance của  data)

Khi ta không biết variance thực sự của data, ta có thể:
1. Chọn một con số thích hợp
2. Tính variance của data rồi áp vào
3. Model variance của data theo phân phối inverse gamma. **bài sẽ nói pp này**

**Công thức:**  
- Giả sử một biến số x:  
$ x \sim N(\mu, \sigma^2) $  
- Prior cho $\mu$:  
$ \mu \sim N(m, \sigma_\mu^2 = \frac{\sigma^2}{w})$ với $ w=\frac{\sigma^2}{\sigma^2_\mu}$  
- Prior cho $\sigma^2$:  
$ \sigma^2 \sim \Gamma^{-1} (\alpha, \beta) $  

Sau khi tính toán, ta có:  
- posterior cho $\sigma^2$:  
$ \sigma^2|X \sim \Gamma^{-1} \left(\alpha +\frac{n}{2}, \beta + \frac{1}{2}\displaystyle\sum_{i=1}^n(X_i -\overline{X})^2 + \frac{nw}{2(n+w)}(\overline{X}-m)^2\right)$   
- posterior cho $\mu$:  
<font size="+1">$ \mu|\sigma^2, X \sim N\left( \frac{n}{n+w}\overline{X} + \frac{w}{n+w}m, \frac{\sigma^2}{n+w}      \right) $</font>  

Để ý mean của posterior $\mu$, ta thấy nó là:  
- data weight x data mean + prior weight x prior mean
- w là effective sample size

Chúng ta có thể lồng ghép cho $\sigma^2$ để posterior $\mu$ không phụ thuộc nó nữa, khi đó $\mu$ sẽ có phân phối t.  
$\mu|X \sim t$( dof, loc, scale)
- dof = $2\alpha + n$ 
- loc = <font size="+1">$\frac{n\overline{X} + wm}{n+w}$</font>
- scale = <font size="+1">$\sqrt{ \frac{ \beta+ \frac{n-1}{2}s^2 + \frac{nw}{2(n+w)}(\overline{X}-m)^2  }{(n+w)(\alpha + n/2)}  }$</font> với $s^2 = \sum(X_i - \overline{X})^2/(n-1) $ là sample variance.

---

In [1]:
# Ví dụ về đo nhiệt độ sôi của nước
# Lấy data là nhiệt độ đo được khi nước bắt đầu sôi bằng thermometer
# data phân phối theo normal (mean = theta, var = 0.25) (đã biết var)
# Giả sữ prior cho theta có mean = 100, var = s^2
# posterior Y|theta = N( theta, 0.25); theta = N( 100, s^2)

In [2]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
%matplotlib inline

In [53]:
data = np.array([94.6, 95.4, 96.2, 94.9, 95.9])

n = len(data)
data_mean = data.mean()
data_sigma2 = 0.25

In [52]:
# tính var của prior
# giả sử sample effective size = 1
# 0.25 / s2 = 1 => s2 = 0.25
prior_s2 = 0.25

prior_mean = 100

In [54]:
# effective sample size = 1 
w = data_sigma2/prior_s2

# posterior_mean
posterior_mean = n/(n+w)*data_mean + w/(n+w)*prior_mean

# posterior var
posterior_var = data_sigma2 / (n+w)

posterior_mean, posterior_var

(96.16666666666669, 0.041666666666666664)

In [57]:
# 95% HPD
st.norm(loc=posterior_mean, scale=np.sqrt(posterior_var)).interval(.95)

(95.76659069363707, 96.5667426396963)

In [8]:
# mật độ xác suất theta < 100
st.norm(loc=5/6 * data.mean() + 1/6*100, scale=np.sqrt(0.25/6)).cdf(100)

# cdf =1 nghĩa là xác suất posterior mean < 100 là 100%. Tức là thermometer bị sai số xuống giá trị thấp

1.0

In [56]:
# posterior predictive distribution
# N(posterior_mean , data_var + posterior_var)
posterior_mean, data_sigma2 + posterior_var

(96.16666666666669, 0.2916666666666667)

---

In [37]:
# Đo trọng lượng bánh của tiệm bánh A
# data phân phối normal không biết mean và variance, n = 27
# trong luong ~ N( mean, sigma^2)
# mean ~ N(m , sigma^2/w), effective sample size w=0.1 do không chắc lắm
# mình dự đoán m = 500
# sigma^2 ~ Inverse Gamma (a=3, b=200); trung bình là 100

# prior mean
m = 500
w = 0.1

# prior sigma^2
a = 3
b = 200

# data
n = 27
x = 609.7
s2 = 401.8

In [59]:
# rvs(n) là sampling n mẫu
prior_sigma_rvs = st.invgamma(a=a, scale=b).rvs(300)
prior_sigma_rvs.mean()

94.83405869378143

In [39]:
# posterior sigma^2
alpha = a + n/2
beta = b + (n-1)/2*s2 + w*n/ ( 2*(w+n)) * (x-m)**2
alpha, beta

(16.5, 6022.884188191883)

In [40]:
# posterior mu 
mu_mean = (n*x + w*m)/(n+w)
mu_mean

609.2952029520295

In [45]:
# sampling posterior mu mean
# đầu tiên sampling posterior sigma
# sau đó cho vào posterior mu rồi sampling

posterior_sigma_rvs = st.invgamma(a=alpha, scale=beta).rvs(1000)

posterior_mu_rvs = st.norm(loc=mu_mean, scale = np.sqrt(posterior_sigma_rvs/(n+w))).rvs(1000)

In [46]:
# 95% CI for posterior mu
np.quantile(posterior_mu_rvs, [0.025, 0.975])

array([601.82015773, 616.48640522])

In [48]:
# data ở một tiệm bánh B
# prior mean
m_B = 500
w_B = 0.1

# prior sigma^2
a_B = 3
b_B = 200

# data
n_B = 30
x_B = 622.8
s2_B = 403.1

In [49]:
alpha_B = a_B + n_B/2
beta_B = b_B + (n_B-1)/2*s2_B + w_B*n_B/ ( 2*(w_B+n_B)) * (x_B-m_B)**2
mu_mean_B = (n_B*x_B + w_B*m_B)/(n_B+w_B)
posterior_sigma_rvs_B = st.invgamma(a=alpha_B, scale=beta_B).rvs(1000)
posterior_mu_rvs_B = st.norm(loc=mu_mean_B, scale = np.sqrt(posterior_sigma_rvs_B/(n_B+w_B))).rvs(1000)
np.quantile(posterior_mu_rvs_B, [0.025, 0.975])

array([615.39433943, 629.0970516 ])

In [50]:
# Null hypothesis: tiệm bánh B tốt hơn tiệm bánh A
# tức là bánh ở tiệm B nhiều hơn bánh ở tiệm A
# lấy n mẫu từ posterior mu ở B so sánh với n mẫu từ posterior mu A. Sau đó cộng lại chia n
np.sum(posterior_mu_rvs_B > posterior_mu_rvs)/1000

0.996

**Xác suất tiệm bánh B cho nhiều hơn tiệm bánh A >95%.**